In [ ]:
Vấn đề cách chúng ta gọi API:
- Services: Nơi để chứa các axios gọi API
- Nhưng nơi dispatch(action) lại ở hooks:
    - Trong hooks lại phải dispatch(action)

? Vậy mỗi khi muốn gọi API nữa phải trả qua nhiều bước cồng kềnh
    1. Tạo một hàm trong services
    2. Vào hook => custom hook => dispatch(action)
    3. Tạo action tương ứng
> Đặt ra câu hỏi: 
    - Có thể gọi API trong actions luôn được không?
    - Và khi gọi thành công thì action đó tự dispatch để set luôn được không?
> Ý là: 
    - Tôi có thể dispatch một action mà action đó gọi API luôn được không? 
    => ngắn hơn và đỡ phải sửa nhiều nơi hơn

In [ ]:
//src/features/product/consts.js

/* 1. Tạo action.type: GET_ITEMS  */
export const GET_ITEMS = `${NAMESPACE}/getItems`;

In [ ]:
//src/features/product/actions.js

//...

/* 2. Action: getItems */
export const getItems = () => {
    // Gọi API ở đây được không?
    /* 
        - Gọi API thì vẫn gọi được:
        - Nhưng vấn đề ở đây là:
            - Vai trò của một action: Hàm tạo action cuối cùng trả ra một object {type, payload}
            => Nếu gọi ở đây kể cả có return object {type, payload} thì hàm đó khi gọi 
                - ❌ sẽ không trả ra object 
                - ✅ trả ra một Promise (khi nó được resolve thì sẽ nhận vào thứ mà chúng ta return)
                => Vì gọi API là một thao tác bất đồng bộ
    */
};


In [ ]:
//src/features/product/actions.js

/* TÌNH HUỐNG HIỂU SAI 
    - Cứ tưởng nếu gọi hàm getItems sẽ return ra response
    - Thực tế gọi hàm sẽ return về Promise có resolve(response)
*/

/* Action: getItems */
export const getItems = async () => {
    const response = await getProducts();
    // Giả sử trả về object
    return {
        type: SET_ITEMS,
        payload: response.data.items,
    };
};


In [ ]:
// src/pages/Home/index.jsx



function Home() {
    const dispatch = useDispatch()

    /* Thử dispatch với action getItems:
        - Kỳ vọng: Dispatch nhận đầu vào là một action là một object {type, payload}
        - Thực tế: getItems(): đầu vào của dispatch lúc này không phải một action mà là một Promise do hàm getItems() là một hàm async (bất đồng bộ)

        - Promise được đưa vào dispatch
        - Promise bị bắn vào reducer
        - Reducer không xử lý được cục promise


        => Lỗi: Actions must be plain objects.
        => Hiểu là: dispatch nhận vào một action mà action đó bắt buộc phải là một object thuần tuý 
        => Bản thân redux sẽ phải kiểm tra đầu vào của dispatch là action không phải là object bình thường => văng lỗi

        
    */
    useEffect(() => {
        console.log(getItems()); // getItem(): Promise
        dispatch(getItems())
    }, [dispatch]);

    return (
        <div>
            {/* ... */}
        </div>
    );
}

export default Home;


In [ ]:
Để giải quyết vấn đề này người ta sử dụng một middleware:
=> redux-thunk

- Bình thường dispatch() chỉ nhận object thuần tuý. Nếu muốn dispatch() một giá trị khác => dùng middleware: [redux-thunk]

In [ ]:
npm i redux-thunk

`Redux-thunk`

In [ ]:
Vai trò: Cho phép dispatch một function. Function đó có thể viết được logic. Và trong function đó nhận 2 tham số là (dispatch, getState) => Lúc này chúng ta sẽ thực hiện dispatch lần 2

=> Có thể chờ async function trả về lấy được response thì lúc đó chúng ta mới thật sự dispatch
=> dispatch 2 lần:
    - Lần 1: dispatch(function) (VD: getItems) cho phép chúng ta viết logic ở bên trong => Gọi API
    - Lần 2: Chờ API phản hồi, có dữ liệu rồi thì mới gọi dispatch lần 2

In [ ]:
// src/store/index.js

/* 1. Bổ sung (Lưu ý không import default => import lẻ {...}) */
import { thunk } from "redux-thunk"; 


/* 2. Push thẳng vào middleware vì trên môi trường PROD cần có thunk */
const enhancers = [thunk];

if (!import.meta.env.PROD) {
    enhancers.push(logger);
}

const store = legacy_createStore(rootReducer, applyMiddleware(...enhancers));
export { store };


In [ ]:
// src/features/product/actions.js

//...


export const getItems = () => {
    /* 1. Trả về một hàm có 2 tham số (dispatch, getState) */
    return (dispatch, getState) => {
        // dispatch: hàm dispatch
        // getState(): trả ra state tổng
    };
};


In [ ]:
//src/pages/Home/index.jsx

function Home() {
    //...

    useEffect(() => {
        dispatch(getItems()); // getItems() lúc này trả về một hàm => dispatch(hàm) => thunk cho phép làm điều đó
    }, [dispatch]);

    //...
}

export default Home;


In [ ]:
/* ? Tại sao lại dispatch(setItems(...) trong hàm getItems):
    Mục tiêu: Bạn muốn lấy danh sách items => get()
        B1: Thực hiện fetch API
        B2: Khi có kết quả, dispatch setItems(items) để store cập nhật (lưu kết quả của fetch vào store)
    => Nói cách khác:
    - setItems chỉ là công cụ hỗ trợ
    - getItems vẫn đúng với ý nghĩa “tôi đang lấy dữ liệu”
*/

/* Action: getItems */
export const getItems = () => {
    // Trả ra một function có 2 tham số (dispatch, getState)
    return async (dispatch, getState) => {
        // Lấy response từ fetch API
        const {
            data: { items },
        } = await getProducts();

        // Cập nhật state với kết quả vừa nhận được
        dispatch(setItems(items));
    };
};

// ? Chúng ta vẫn có thể lấy dispatch = store.dispatch vẫn được nhưng không nên.
// => Vì store nên chỉ được import ở một nơi

In [ ]:
// Xoá toàn bộ code trong src/services/product/hooks.js

In [ ]:
/* 
Nhận ra một điều:
Tại: src/pages/Home/index.jsx
=> dispatch(getItems()); // Logger action của setItems()
=> Logger của setItems sinh ra từ dispatch(setItems)
? Vậy tại sao dispatch(getItems()) lại không sinh ra logger
> Bởi vì redux-thunk sinh ra để cho chúng ta dispatch với action là một hàm để chúng ta viết logic bên trong.
> Khi nó kiểm tra thấy cái action là một hàm thì nó KHÔNG THỰC HIỆN dispatch HÀM ĐÓ để đẩy vào reducer. Khi thấy điều đó nó sử dụng middleware để xử lý => Vậy chỉ logger khi dispatch(action) action phải là một object thuần tuý.
> LƯU Ý: redux-thunk chỉ gọi hàm nếu:
    + action là một HÀM
    + HÀM đó phải là hàm dạng thunk: nhận (dispatch) => {...}

> Xử lý loading, phải biết được thời điểm nó get (gọi API 
=> Xử lý trước khi gọi API
=> để dispatch một action
=> reducer xử lý isLoading
*/

In [ ]:
// src/features/product/action.js

export const getItems = () => {
    return async (dispatch) => {
        // Trước khi gọi API
        /* Xử lý loading với dispatch action thuần tuý */
        dispatch({ type: GET_ITEMS });

        // Gọi API
        const {
            data: { items },
        } = await getProducts();

        // Sau khi có response
        dispatch(setItems(items));
    };
};


In [ ]:
// src/features/product/reducer.js

const initState = {
    items: [],
    isLoading: false, // Bổ sung state isLoading: Để chúng ta biết product có đang loading hay không?
};

function reducer(state = initState, action) {
    switch (action.type) {
        // Thêm case để xử lý loading trước khi gọi API
        case GET_ITEMS:
            return {
                ...state,
                isLoading: true,
            };
        case SET_ITEMS:
            return {
                ...state,
                items: action.payload,
                // Sau khi gọi API => cập nhật state => tắt loading
                isLoading: false,
            };
        default:
            return state;
    }
}

export default reducer;



In [ ]:
//src/pages/Home/index.jsx

function Home() {
    const dispatch = useDispatch();
    /* Lấy state */
    const { items: products, isLoading } = useSelector(
        (state) => state.product,
    );
    console.log(products, isLoading);

    useEffect(() => {
        dispatch(getItems());
    }, [dispatch]);

    return (
        <div>
            <h1>Home</h1>
            <h2>Product List</h2>
            {/* Hiển thị lên giao diện */}
            <ul>
                {isLoading ? (
                    <div>Loading...</div>
                ) : (
                    products.map((product) => (
                        <li key={product.id}>
                            {product.id}.{product.title} - {product.price}
                        </li>
                    ))
                )}
            </ul>
        </div>
    );
}

export default Home;


TÓM TẮT MỘT SỐ KHÁI NIỆM

1. Action
    - Là một object thuần tuý
    - Reducer nhận được action này để xử lý
    - Reducer chỉ làm việc với object thuần tuý

In [ ]:
{ type: "GET_ITEMS" }

2. Action Creator (tầng 1)
    - Là một HÀM để tạo ra ACTION
    - Nó chỉ return ra một thứ (object hoặc function)

In [ ]:
// return một object
function getItems() {
    return { type: "GET_ITEMS" };
}

// return một hàm thunk function
function getItems() {
    return (dispatch) => {
        //...
    };
}

3. Thunk Function (tầng 2)
- Là function mà Redux-thunk thực sự thực thi
- Là function nhận vào `dispatch`
- Chạy logic async ở bên trong
- Tự dispatch các action object thật


In [ ]:
(dispatch) => {
    dispatch({ type: GET_ITEMS });
    const data = await fetch()
    dispatch({ type: SET_ITEMS, payload: data })
}


Thunk function được gọi khi: (Middleware can thiệp)
- Nếu dispatch nhận vào một function → ta sẽ gọi nó → truyền dispatch vào
- Nếu dispatch nhận object → ta sẽ gửi tới reducer

In [ ]:
// dispatch đi qua middleware redux-thunk
// Redux-thunk kiểm tra: => nếu là thunk function => gọi hàm
if (typeof action === "function") { 
    return action(dispatch, getState)
}
